In [ ]:
# Parameters

# select test(1) or training(0)
test = 0

batch_size = 1
number_of_epochs = 100
filename = '_defEx_InspMask2' #'_DIV_OK' # _DIV_OK

# save csv metrics and disease maps
save = 0

In [ ]:
cd /data-synology/anlee/

In [ ]:
ls

In [ ]:
# includes&imports
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1" #for CPU ""

import shutil
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
#import nilearn as nil
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

# random seeds
np.random.seed(16)
tf.random.set_seed(16)
tf.keras.utils.set_random_seed(16)

Build an input pipeline with image paths

In [ ]:
# get image pathes for input and target images
from tqdm import tqdm

root_directory = '/data-synology/anlee/COPDGene/'  # Replace with the actual path to your root directory
inputImageName = ('insp_ct_ds.nii') #('insp_ct.nii')
outputImageName = ('exp_ct_deform_ds.nii')  # Extensions of the target image files
maskInspImageName = ('insp_mask_cat.nii')  # Mask image
#maskDefExpImageName = ('exp_mask_deform_cat.nii')  # Mask image


def search_images(directory, image_list, name):
  for root, dirs, files in tqdm(os.walk(directory)):
        for file in files:
            if file.endswith(name):
                image_path = os.path.join(root, file)
                image_list.append(image_path.replace('\0', ''))  # Add the image path to the list, replace termination character

# Create an empty list to store image paths
inputImagePath = []
outputImagePath = []
maskInspImagePath = []
#maskDefExpImagePath = []

# Call the search_image function with the root directory
search_images(root_directory, inputImagePath, inputImageName)
search_images(root_directory, outputImagePath, outputImageName)
search_images(root_directory, maskInspImagePath, maskInspImageName)
#search_images(root_directory, maskDefExpImagePath, maskDefExpImageName)

# check if path loaded
if inputImagePath:
    print("Image input paths:")
    for path in inputImagePath[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")

if outputImagePath:
    print("Image ouput paths:")
    for path in outputImagePath[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")
    
if maskInspImagePath:
    print("Image mask insp paths:")
    for path in maskInspImagePath[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")
    
#if maskDefExpImagePath:
#    print("Image mask defExp paths:")
#    for path in maskDefExpImagePath[:5]:
#        print(path)
#else:
#    print("No image files found in the directory tree.")

In [ ]:
from sklearn.model_selection import train_test_split

# Convert image paths to lists
inputImagePath = list(inputImagePath)
outputImagePath = list(outputImagePath)
maskInspImagePath = list(maskInspImagePath)
#maskDefExpImagePath = list(maskDefExpImagePath)

# Split the data into training and test sets list
train_input, test_input, train_output, test_output, train_insp_mask, test_insp_mask = train_test_split(
    inputImagePath, outputImagePath, maskInspImagePath, test_size=0.3, random_state=42)

# Split the data into test and validation sets list
val_input, test_input, val_output, test_output, val_insp_mask, test_insp_mask = train_test_split(
    test_input, test_output, test_insp_mask, test_size=0.5, random_state=42)

print('training data: ' + str(len(train_input)))
print('validation data: ' + str(len(val_input)))
print('test data: ' + str(len(test_input)))
print('train mask inspiratory data: ' + str(len(train_insp_mask)))
print('test mask inspiratory data: ' + str(len(test_insp_mask)))
#print('train mask expiratory data: ' + str(len(train_exp_mask)))
#print('test mask expiratory data: ' + str(len(test_exp_mask)))

if train_input:
    print("Image input paths:")
    for path in train_input[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")

if train_output:
    print("Image ouput paths:")
    for path in train_output[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")
    
if train_insp_mask:
    print("Image mask inspiratory paths:")
    for path in train_insp_mask[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")


In [ ]:
# image function to load images
def load_image_scale(file_path):
    
    # load nibable image
    image_data = nib.load(file_path).get_fdata()

    # pick middle slice
    image = image_data

    # convert values
    image = (image / 3000)

    return image

In [ ]:
# image function to load images
def load_image(file_path):

    # load nibable image
    image_data = nib.load(file_path).get_fdata()
    
    # pick middle slice
    image = image_data
    
    return image

In [ ]:
def load_image_mask(file_path):

    # load nibable image
    image_data = nib.load(file_path).get_fdata()

    # pick middle slice
    mask = image_data

    # create binary image mask
    binary_mask_slice = np.where((mask > 0) & (mask < 6), 1, 0)
    
    return binary_mask_slice

In [ ]:
# select test or train
if test == 1:
    test_input_smaller = test_input[0::128]
    test_output_smaller = test_output[0::128] 
    test_mask_insp_smaller = test_insp_mask[0::128]


    test_dataset_paths = (test_input_smaller, test_output_smaller)
    test_mask_insp_paths = test_mask_insp_smaller

    
else:
    train_dataset_paths = (train_input, train_output)

    test_dataset_paths = (test_input, test_output)
    test_mask_insp_paths = test_insp_mask

# calucalte steps and so on
length = len(test_dataset_paths[0])
print('length test:')
print(length)

# calculate steps per epoch and validation steps
steps_per_epoch = len(test_dataset_paths[0]) // batch_size
print('steps_per_epoch: ' + str(steps_per_epoch))

test_steps = len(test_dataset_paths[0]) // batch_size
print('test_steps: ' + str(test_steps))

# calculate the number of training iterations
number_of_steps_total = int(steps_per_epoch * number_of_epochs)
print('number_of_steps_total: ' + str(number_of_steps_total))


In [ ]:
from random import shuffle

def data_generator(paths, mask_insp_paths, batch_size):
    test_input_paths, test_output_paths = paths
    
    num_samples = len(test_input_paths)
    #print(num_samples)
    indices = list(range(num_samples))
        
    for i in range(0, num_samples, batch_size):
        batch_indices = indices[i:i+batch_size]

        #print(batch_indices)
        
        test_input_images = []
        test_output_images = []
        test_mask_images = []

        
        for idx in batch_indices:
            # load mask
            mask = load_image_mask(mask_insp_paths[idx])
            # load input image
            input_image = load_image_scale(test_input_paths[idx])
            # mask input image
            input_image = input_image * mask
            # load deformed exp image
            defexp_image = load_image(test_output_paths[idx])

            # calculate subtraction and mask it
            output_image = defexp_image * mask

            # append images to input and output train
            test_input_images.append(np.expand_dims(input_image, -1))
            test_output_images.append(np.expand_dims(output_image, -1))
            test_mask_images.append(np.expand_dims(mask, -1))
        
        yield np.array(test_input_images), np.array(test_output_images), np.array(test_mask_images)


Image generator


In [ ]:
# create data generators for test and validation
test_generator = data_generator(test_dataset_paths, test_mask_insp_paths, batch_size)

In [ ]:
returned_values = next(test_generator)
print(np.array(returned_values).shape)

In [ ]:
#inp, out = next(test_generator)
#print(inp.shape, out.shape)
print(number_of_steps_total)
print(batch_size)

In [ ]:
number_of_steps_total = 638000

In [ ]:
# load model HERE WE ARE NOW
from tensorflow.keras.models import load_model

model_path = f'/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/'

model = load_model(model_path)
print('Model loaded')

In [ ]:
# Calcualte MSE
def compute_mse(ground_truth, prediction):
    mse = np.mean((ground_truth - prediction) ** 2)
    return mse

In [ ]:
# calcualte airtrapping
def compute_airtrapping_per_image(inspiratory, defExp_prediction_image, mask):

    subtraction_image = (defExp_prediction_image - inspiratory) * mask
    
    sub_prediction_image = subtraction_image * 3000

    threshold = 100

    # count values below threshold
    airtrapping_voxels = np.sum((sub_prediction_image < threshold) & (sub_prediction_image != 0))
    
    # count total total values
    total_voxels = np.count_nonzero(sub_prediction_image)

    # calcualte percentage airtrapping
    percentage_airtrapping = (airtrapping_voxels / total_voxels) * 100
    
    return percentage_airtrapping

In [ ]:
# Dice score
# input: deformed expiratory, inpsriatory, predicted
def calculate_dice_score(inspiratory, output, masked_predictions, mask):

    threshold = 100
    
    # defExp - inspiratory
    sub_real = (output - inspiratory)
    mt = np.where(sub_real * 3000 < threshold, 1, 0) * mask

    # defExp predict - inspiratory
    sub_fake = (masked_predictions - inspiratory)
    mp = np.where(sub_fake * 3000 < threshold, 1, 0) * mask
    
    dice_value = (2 * np.sum(mt * mp)) / (np.sum(mt + mp))
    
    return dice_value

In [ ]:
# Initialize placeholders for statistics

mse_values = []
airtrapping_percentages_fake = []
airtrapping_percentages_real = []
dice_scores = []

# Initialize placeholders for worst case tracking
worst_slice_idx = None
worst_slice_value = float('inf')  # Assuming you want to minimize the metric
worst_slice_batch_idx = None

# Loop through generated batches, batch input: inspiratory, batch output: subtraction
for batch_input, batch_output, batch_masks in tqdm(test_generator, total=test_steps, desc="Processing batches"):

    # predict
    predictions = model(batch_input, training=False)

    masked_predictions = []
    
    # maske predictions
    for prediction, mask in zip(predictions,batch_masks):
        mask_pred = prediction * mask
        masked_predictions.append(mask_pred)

    # calculate MSE
    for defExp, prediction in zip(batch_output, masked_predictions):
        mse_values.append(compute_mse(defExp, prediction))

    # Iterate through the lists and calculate airtrapping percentages
    for inspiratory, prediction, mask in zip(batch_input, masked_predictions, batch_masks):
        airtrapping_percentages_fake.append(compute_airtrapping_per_image(inspiratory, prediction, mask))

    # Iterate through the lists and calculate airtrapping percentages
    for inspiratory, subtraction, mask in zip(batch_input, batch_output, batch_masks):
        airtrapping_percentages_real.append(compute_airtrapping_per_image(inspiratory, subtraction, mask))

    # Iterate through the lists and calculate dice scores
    for inspiratory, output, predictions, mask in zip(batch_input, batch_output, masked_predictions, batch_masks):
        dice_scores.append(calculate_dice_score(inspiratory, output, predictions, mask))


In [ ]:
from tqdm import tqdm

# Assume test_generator, model, compute_mse, and calculate_dice_score are defined elsewhere
# Assume compute_airtrapping_per_image can be adapted for 2D slices as needed

# Initialize placeholders for statistics
mse_values = []
airtrapping_percentages_fake = []
airtrapping_percentages_real = []
dice_scores = []

# Initialize placeholders for worst case tracking
worst_slice_idx = None
worst_slice_value = float('inf')  # Assuming you want to minimize the metric
worst_slice_batch_idx = None
worst_slice_image_idx = None  # To track the image within the batch

# Initialize a list to store detailed information including differences for each slice
airtrapping_differences = []

# Loop through batches in the generator
for batch_idx, (batch_input, batch_output, batch_masks) in enumerate(tqdm(test_generator, total=test_steps, desc="Processing batches")):
    predictions = model(batch_input, training=False)

    for item_idx in range(batch_input.shape[0]):  # Iterate over items in the batch
        image_details = {
            'batch_idx': batch_idx, 
            'item_idx': item_idx, 
            'worst_difference': 0, 
            'worst_slice_idx': None, 
            'slice_details': []
        }
        
        for slice_idx in range(batch_input.shape[1]):  # Iterate over slices in the depth dimension
            inspiratory_slice = batch_input[item_idx, slice_idx, :, :]
            prediction_slice = predictions[item_idx, slice_idx, :, :]
            mask_slice = batch_masks[item_idx, slice_idx, :, :]
            
            # Calculate true and predicted air trapping percentages
            true_airtrapping = compute_airtrapping_per_image(inspiratory_slice, batch_output[item_idx, slice_idx, :, :], mask_slice)
            predicted_airtrapping = compute_airtrapping_per_image(inspiratory_slice, prediction_slice, mask_slice)
            
            # Compute the difference between true and predicted air trapping
            difference = abs(true_airtrapping - predicted_airtrapping)
            
            # Store slice details including true and predicted air trapping values
            image_details['slice_details'].append({
                'slice_idx': slice_idx,
                'true_airtrapping': true_airtrapping,
                'predicted_airtrapping': predicted_airtrapping,
                'difference': difference
            })
            
            # Update the worst slice for the current image based on the greatest difference
            if difference > image_details['worst_difference']:
                image_details['worst_difference'] = difference
                image_details['worst_slice_idx'] = slice_idx
                # Also store true and predicted air trapping values for the worst slice
                image_details['worst_true_airtrapping'] = true_airtrapping
                image_details['worst_predicted_airtrapping'] = predicted_airtrapping

        # Append the current image's details to the list
        airtrapping_differences.append(image_details)

In [ ]:
# Sort images by their worst difference in descending order
sorted_images = sorted(airtrapping_differences, key=lambda x: x['worst_difference'], reverse=True)

# Extract the worst 20 images
worst_20_images = sorted_images[:100]

# For each of the worst 20 images, identify and report the worst slice
for image in worst_20_images:
    print(f"Batch {image['batch_idx']}, Image {image['item_idx']}: Worst Slice {image['worst_slice_idx']} with Difference {image['worst_difference']}")


In [ ]:
test_generator4 = data_generator(test_dataset_paths, test_mask_insp_paths, batch_size)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming test_generator2 is a predefined generator that can be reset or iterated from the beginning
# Reset or reinitialize your generator as needed
test_generator2.reset()

# Placeholder for the current batch index during iteration through the generator
current_batch_idx = -1

# Iterate through the generator to find and plot the worst slices
for batch_input, batch_output, batch_masks in test_generator4:
    current_batch_idx += 1
    
    # Filter for images from the current batch
    filtered_images = [img for img in worst_20_images if img['batch_idx'] == current_batch_idx]
    
    # If there are no images from this batch in the worst 20, continue to the next batch
    if not filtered_images:
        continue
    
    for image_details in filtered_images:
        item_idx = image_details['item_idx']
        worst_slice_idx = image_details['worst_slice_idx']
        worst_true_airtrapping = image_details['worst_true_airtrapping']
        worst_predicted_airtrapping = image_details['worst_predicted_airtrapping']
        worst_difference = image_details['worst_difference']
        worst_slice_mse = image_details['worst_slice_mse']
        worst_slice_dice_score = image_details['worst_slice_dice_score']

        # Generate predictions for the current batch
        predictions = model.predict(batch_input)
        
        # Process the specific worst slice for visualization
        inspiratory_slice = batch_input[item_idx, worst_slice_idx, :, :] * 3000
        prediction_slice = predictions[item_idx, worst_slice_idx, :, :] * 3000
        ground_truth_slice = batch_output[item_idx, worst_slice_idx, :, :] * 3000

        # Use np.squeeze and np.rot90 for simplification
        insp_slice_2d = np.rot90(np.squeeze(inspiratory_slice))
        pred_slice_2d = np.rot90(np.squeeze(prediction_slice))
        gt_slice_2d = np.rot90(np.squeeze(ground_truth_slice))

        # Visualization
        plt.figure(figsize=(12, 6))

        # Display the ground truth slice
        plt.subplot(1, 3, 1)
        plt.imshow(insp_slice_2d, cmap='gray')
        plt.title('Inspiratory Slice')
        plt.axis('off')

        # Display the prediction slice
        plt.subplot(1, 3, 2)
        plt.imshow(pred_slice_2d, cmap='gray')
        plt.title('Prediction Slice')
        plt.axis('off')

        # Display the ground truth slice for comparison
        plt.subplot(1, 3, 3)
        plt.imshow(gt_slice_2d, cmap='gray')
        plt.title('Ground Truth Slice')
        plt.axis('off')

        # Annotations for air trapping, MSE, and Dice scores
        plt.suptitle(f"Batch {image_details['batch_idx']}, Image {image_details['item_idx']}, Slice {worst_slice_idx}\n" +
                     f"True Air Trapping: {worst_true_airtrapping:.2f}, Predicted Air Trapping: {worst_predicted_airtrapping:.2f}, " +
                     f"Difference: {worst_difference:.2f}\nMSE: {worst_slice_mse:.4f}, Dice Score: {worst_slice_dice_score:.4f}",
                     fontsize=14)

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()


In [ ]:
for idx, mse in enumerate(mse_values[:5]):
    print(f"MSE at position {idx}:", mse)

In [ ]:
for idx, airtrapping in enumerate(airtrapping_percentages_fake[:5]):
    print(f"Airtrapping percentage for Prediciton {idx}: {airtrapping}%")

In [ ]:
for idx, airtrapping in enumerate(airtrapping_percentages_real[:5]):
    print(f"Airtrapping percentage for Subtraction {idx}: {airtrapping}%")

In [ ]:
for idx, dicescore in enumerate(dice_scores[:5]):
    print(f"Airtrapping percentage for Subtraction {idx}: {dicescore}%")

In [ ]:
import pandas as pd
df_metrics = pd.DataFrame(columns=["Predicted_Image", "MSE", "Air-trp 1 [%]", "Air-trp 2 [%]", "Dice Score"])

In [ ]:
# add values to metric dataframe
for i, (mse, airtrapping, airtrapping_check, dice) in enumerate(zip(mse_values, airtrapping_percentages_fake, airtrapping_percentages_real, dice_scores)):
    df_metrics.loc[i] = [i, mse, airtrapping_check, airtrapping, dice]

In [ ]:
print(df_metrics[:50])

In [ ]:
if save == 1:
    try:
        os.mkdir(f"/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/test/")
        os.mkdir(f"/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/test/metrics/")
        os.mkdir(f"/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/test/plots/")
    
    except Exception as e:
        print("An error occurred:", str(e))

In [ ]:
# save to csv
if save == 1:
    df_metrics.to_csv(f'/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/test/metrics/metrics.csv', index=False)

In [ ]:
# plot metrics of df
mean_mse = df_metrics['MSE'].mean()
std_mse = df_metrics['MSE'].std()

mean_dice = df_metrics['Dice Score'].mean()
std_dice = df_metrics['Dice Score'].std()

# Represent in the format "mean ± SD"
mse_str = f"{mean_mse:.2e} ± {std_mse:.2e}"
dice_str = f"{mean_dice:.2f} ± {std_dice:.2f}"

print(f'MSE: {mse_str}')
print(f'Dice Score: {dice_str}')

In [ ]:
import seaborn as sns

# Set the style and size
sns.set_style("whitegrid")
plt.figure(figsize=(10, 8))

# Scatter plot
sns.scatterplot(x=df_metrics["Air-trp 1 [%]"], y=df_metrics["Air-trp 2 [%]"], alpha=0.6)

# Title and labels
plt.title("Correlation between Air-trapping 1 (True) and Air-trapping 2 (Prediction)", weight='bold')
plt.xlabel("Air-trapping 1 [%] (True)")
plt.ylabel("Air-trapping 2 [%] (Prediction)")

# Adding the identity line
limits = [min(plt.xlim()[0], plt.ylim()[0]), max(plt.xlim()[1], plt.ylim()[1])]
plt.plot(limits, limits, 'r-', label='Identity Line')
plt.legend()
if save == 1:
    plt.savefig(f"/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/test/plots/Correlation_nsteps{number_of_steps_total}_batch{batch_size}{filename}.png")
plt.show()

In [ ]:
# The Bland-Altman plot, also known as a difference plot, is used to visualize the agreement
# between two methods or two measurements. It plots the difference between the two measures
# against their average.

# Bland-Altman data
average = (df_metrics["Air-trp 1 [%]"] + df_metrics["Air-trp 2 [%]"]) / 2
difference = df_metrics["Air-trp 1 [%]"] - df_metrics["Air-trp 2 [%]"]

mean_diff = difference.mean()
std_diff = difference.std()

# Set the style and size
sns.set_style("whitegrid")
plt.figure(figsize=(10, 8))

# Scatter plot
sns.scatterplot(x=average, y=difference, alpha=0.6)

# Add mean and limits of agreement lines
plt.axhline(mean_diff, color='red', linestyle='--', label=f'Mean diff: {mean_diff: .2f}')
plt.axhline(mean_diff + 1.96*std_diff, color='blue', linestyle='--', label='Mean diff + 1.96*SD')  #95%
plt.axhline(mean_diff - 1.96*std_diff, color='blue', linestyle='--', label='Mean diff - 1.96*SD')

# Adding mean values next to the lines using plt.text
x_position = max(average)
plt.text(x_position, mean_diff + 1.96*std_diff, f'+1.96 SD: {mean_diff + 1.96*std_diff:.2f}', verticalalignment='bottom', horizontalalignment='right', color='blue')
plt.text(x_position, mean_diff - 1.96*std_diff, f'-1.96 SD: {mean_diff - 1.96*std_diff:.2f}', verticalalignment='bottom', horizontalalignment='right', color='blue')

# Title and labels
plt.title("Bland-Altman Plot between Air-trapping 1 (True) and Air-trapping 2 (Prediction)", weight='bold')
plt.xlabel("Average of Air-trapping 1 [%] (True) and Air-trapping 2 [%] (Prediction)")
plt.ylabel("Difference between Air-trapping 1 [%] (True) and Air-trapping 2 [%] (Prediction)")
plt.legend()

if save == 1:
    plt.savefig(f"/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/test/plots/Bland_Altman_nsteps{number_of_steps_total}_batch{batch_size}{filename}.png")
    
plt.show()


In [ ]:
# ICC
import pingouin as pg

# Reshape dataframe for ICC calculation
df_melted = df_metrics.melt(id_vars=['Predicted_Image'], value_vars=['Air-trp 1 [%]', 'Air-trp 2 [%]'], 
                    var_name='Method', value_name='Measurement')

# Calculate ICC
icc = pg.intraclass_corr(data=df_melted, targets='Predicted_Image', raters='Method', ratings='Measurement').round(3)

print(icc)

# Print the ICC value for ICC1
print(icc[icc['Type'] == 'ICC1']['ICC'].values[0])

icc_val = float(f"{icc[icc['Type'] == 'ICC1']['ICC'].values[0]:.2f}")


In [ ]:
# table of final statistics

In [ ]:
print(f'MSE: {mse_str}')
print(f'ICC: {icc_val}')
print(f'Dice Score: {dice_str}')

In [ ]:
results_df = pd.DataFrame({
    'Metric': ['MSE', 'ICC', 'Dice Score'],
    'Value': [mse_str, icc_val, dice_str]
})

print(results_df)

In [ ]:
# filter worst cases
df_sorted_asce = df_metrics.sort_values(by='Dice Score', ascending=True)
print(df_sorted_asce[:20])

In [ ]:
# create array with worst 20 cases wioth id
predicted_image_ids = np.array(df_sorted_asce['Predicted_Image'][:20])

# Now predicted_image_ids contains the IDs of interest
print(predicted_image_ids)

In [ ]:
if save == 1:
    results_df.to_csv(f'/data-synology/tkeller/Outputs/3Dnsteps{number_of_steps_total}_batch{batch_size}/test/model_results.csv', index=False)

In [ ]:
# save only disease map
save2 = 0

In [ ]:
if save2 == 1:
    try:
        os.mkdir(f'/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/DiseaseMap/')
    except:
        print('Couldnt create directory')

In [ ]:
test_generator2 = data_generator(test_dataset_paths, test_mask_insp_paths, batch_size)

In [ ]:
from scipy.ndimage import gaussian_filter
from matplotlib import cm
from matplotlib.colors import ListedColormap

desired_indices = predicted_image_ids  # Specify the indices of the images you want
count = 0  # Initialize the counter

# Iterate through the generator
for batch_input, batch_output, batch_masks in test_generator2:
    #print(f'Batch {count} processed')
    for element in desired_indices:
        int_element = np.floor(element).astype(int)
        print(f'Processing index {int_element}')
        
        predictions = model(batch_input, training=False)
        masked_predictions = []

        for prediction, mask in zip(predictions, batch_masks):
            mask_pred = prediction * mask
            masked_predictions.append(mask_pred)

        # Process and plot the first image in each batch (or adjust as needed)
        j = 0  # This assumes you're interested in the first image of each batch
    
        # Select images for creating disease map
        insp_ct_ds = (batch_input[j] * 3000)[96] # masked_inspiratory[j] * 3000
        exp_ct_deform_ds = (batch_output[j] - batch_input[j])[96] # masked_subtraction[j]
        prediction_sub = (masked_predictions[j] - batch_input[j])[96]# masked_predictions[j]
        insp_whole = (batch_masks[j])[96] # np.where((test_mask_images[j] > 0) & (test_mask_images[j] < 6), 1, 0)
        
        # Remove the singular third dimension
        insp_ct_ds_2d = np.rot90(np.squeeze(insp_ct_ds), k=-1)
        exp_ct_deform_ds_2d = np.rot90(np.squeeze(exp_ct_deform_ds), k=-1)
        sub_ct_deform_pred_2d = np.rot90(np.squeeze(prediction_sub), k=-1)
        insp_whole_2d = np.rot90(np.squeeze(insp_whole), k=-1)
        
        # Calculate difference
        insp_slc_mask_smooth = gaussian_filter(insp_ct_ds_2d, sigma=1) * insp_whole_2d
        diff_slc_mask_smooth = np.abs(gaussian_filter(exp_ct_deform_ds_2d * 3000, sigma=1)) * insp_whole_2d
        diff_slc_pred_mask_smooth = np.abs(gaussian_filter(sub_ct_deform_pred_2d * 3000, sigma=1)) * insp_whole_2d
        
        # Flip images
        insp_slc = np.flipud(np.fliplr(insp_ct_ds_2d))
        insp_mask = np.flipud(np.fliplr(insp_whole_2d))
        insp_slc_mask = np.flipud(np.fliplr(insp_slc_mask_smooth))
        diff_slc_pred_mask = np.flipud(np.fliplr(diff_slc_pred_mask_smooth))  # prediction
        diff_slc_mask = np.flipud(np.fliplr(diff_slc_mask_smooth))  # ground truth

        # Red parts with emphysema
        insp_map = cm.get_cmap('Reds', 512)(np.linspace(1,0,512))
        insp_map[:,3] = np.linspace(1,0,512)
        insp_map[:,0:3] = insp_map[256,0:3]
        insp_map=ListedColormap(insp_map)
        
        # Blue parts with air-trapping
        diff_map = cm.get_cmap('Blues', 512)(np.linspace(1,0,512))
        diff_map[:,3] = np.linspace(1,0,512)
        diff_map[:,0:3] = diff_map[128,0:3]
        diff_map=ListedColormap(diff_map)
        
        # Create the figure
        plt.figure(figsize=(20, 10))
        
        # Display the first image on the left side
        plt.subplot(1, 2, 1)
        plt.imshow(insp_slc, cmap='gray', vmin=-400-1500, vmax=-400+1500, origin='lower')
        plt.imshow(np.ma.masked_where(insp_mask == 0, diff_slc_mask), cmap=diff_map, vmin=0, vmax=100, alpha=0.5)
        plt.imshow(np.ma.masked_where(insp_mask == 0, insp_slc_mask), cmap=insp_map, vmin=-1000, vmax=-925, alpha=0.5)
        plt.title(f"Ground-Truth Index {int_element}", fontweight='bold', fontsize=20)
        plt.axis('off')
        
        # Display the second image on the right side
        plt.subplot(1, 2, 2)
        plt.imshow(insp_slc, cmap='gray', vmin=-400-1500, vmax=-400+1500, origin='lower')
        plt.imshow(np.ma.masked_where(insp_mask == 0, diff_slc_pred_mask), cmap=diff_map, vmin=0, vmax=100, alpha=0.5)
        plt.imshow(np.ma.masked_where(insp_mask == 0, insp_slc_mask), cmap=insp_map, vmin=-1000, vmax=-925, alpha=0.5)
        plt.title(f"Prediction Index {int_element}", fontweight='bold', fontsize=20)
        plt.axis('off')
        
        plt.tight_layout()
        if save2 == 1:
            plt.savefig(f'/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/DiseaseMap/DiseaseMap_defEx_{count}.png')
        plt.show()
        plt.close()
        #print(int_element

    # Increment the counter after each batch
    count += 1 

    if count > len(desired_indices):
        break  # Exit loop after the highest index is surpassed

In [ ]:
# import nibabel as nib
# aff = np.array([[0,-1,0,0],[-1,0,0,0],[0,0,1,0],[0,0,0,1]])
# img = nib.Nifti1Image(np.float32(img), aff)
# nib.save(img, 'filename.nii')

In [ ]:
test_generator3 = data_generator(test_dataset_paths, test_mask_insp_paths, batch_size)

In [ ]:
# Attempt to extract a single batch from the generator
try:
    batch_input, batch_output, batch_masks = next(test_generator3)
    # Generate predictions for this batch (assuming 'model' is already defined and compiled)
    predictions = model.predict(batch_input)
    print(f"Batch shapes -- Input: {batch_input.shape}, Output: {batch_output.shape}, Masks: {batch_masks.shape}, Predictions: {predictions.shape}")
except Exception as e:
    print(f"Error extracting batch from generator or generating predictions: {e}")


In [ ]:

def save_nifti_with_affine(batch_input, batch_output, predictions, mask, index, output_dir):
    """
    Saves 3D images with a specific affine transformation as NIfTI files.

    Parameters:
    - batch_input: 4D numpy array of input images [batch_size, depth, height, width].
    - batch_output: 4D numpy array of output images [batch_size, depth, height, width].
    - predictions: 4D numpy array of model predictions [batch_size, depth, height, width].
    - mask: 4D numpy array of masks [batch_size, depth, height, width].
    - index: int, index of the image in the batch to save.
    - output_dir: str, directory to save the NIfTI files.
    """
    # Affine matrix for flipping x and y axes
    affine = np.array([[0, -1, 0, 0],
                       [-1, 0, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]])

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Function to save a single 3D image as a NIfTI file
    def save_image(data, filename):
        # Convert data to float32 as recommended
        data_float32 = np.float32(data)
        # Create and save the NIfTI image
        img = nib.Nifti1Image(data_float32, affine)
        nib.save(img, filename)

    # Save each image
    save_image(batch_input[index], os.path.join(output_dir, f'input_{index}.nii'))
    save_image(batch_output[index] - batch_input[index], os.path.join(output_dir, f'output_deformation_{index}.nii'))
    save_image(predictions[index] - batch_input[index], os.path.join(output_dir, f'prediction_deformation_{index}.nii'))
    save_image(mask[index], os.path.join(output_dir, f'mask_{index}.nii'))

# Example usage
#batch_index = 0  # Index of the image in the batch you want to save
#output_dir = f'/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/DiseaseMap/'  # Specify your output directory here

# Replace the following with the actual data loading code
# batch_input, batch_output, predictions, batch_masks = load_your_data()

# Call the function with your data and desired index
#save_nifti_with_affine(batch_input, batch_output, predictions, batch_masks, batch_index, output_dir)

In [ ]:
import nibabel as nib

# Specify the index of the image you want to save, 0 for the first image
batch_index = 0

# Specify your output directory here, ensure it exists or you have permissions to write to it
output_dir = f'/data-synology/tkeller/Outputs/3D_nsteps{number_of_steps_total}_batch{batch_size}/DiseaseMap/'

# Now call the function with the first image data
save_nifti_with_affine(batch_input, batch_output, predictions, batch_masks, batch_index, output_dir)